In [6]:
import pandas as pd
import re
raw_data = pd.read_csv("../raw/global_warming/Stance dataset.csv")
affirming_devices = ['uncover', 'realize', 'know', 'understand', 'learn', 'concede',
'remember', 'recall', 'discover', 'show', 'reveal', 'see',
'forget', 'find', 'point out', 'indicate', 'acknowledge',
'admit', 'realize', 'notice', 'certify', 'verify', 'corroborate', 'affirm', 'confirm', 'agree', 'conclude',
'proven', 'settled', 'conclusive', 'definitive',
'famed', 'unequivocal', 'skilful', 'notable', 'strong', 'famous', 'Nobel', 'skillful',
'Nobelist', 'Nobel Laureate', 'Nobel prize winner',
'Nobel prize winning', 'prize winning', 'award',
'winning', 'distinguished', 'well-grounded', 'esteemed', 'proficient', 'key', 'evidence', 'noted', 'top',
'preeminent', 'breakthrough', 'significant', 'intelligent', 'of import', 'celebrated', 'novel', 'recent',
'major', 'landmark', 'important', 'distinguished',
'renowned', 'peer-reviewed', 'expert', 'leading',
'thousand', '1000', 'hundred', '100', 'unanimous', 'diverse',
'substantial', 'many', 'multiple', 'dozen', 'numerous']
escaped_terms = [re.escape(term) for term in affirming_devices]
pattern = '|'.join(escaped_terms)
to_label = {"A" : 1, "B" : 0, "C" : 0, "agrees": 1, "neutral" : 0, "disagrees": 0}
raw_data

,Unnamed: 0.1,Unnamed: 0,MACE_pred,av_rating,sentence,worker_0,worker_1,worker_2,worker_3,worker_4,...,sent_id,disagree,agree,neutral,confidence_in_prediction_gpt-3.5,confidence_in_prediction_gpt-4o,label_gpt4o,label_gpt-3.5,3.5,4
0,0,0,disagrees,-1.000,Global warming is a hoax.,disagrees,disagrees,disagrees,disagrees,disagrees,...,s0,NaN,NaN,NaN,1.0,1.00,C,C,disagrees,disagrees
1,1,1,neutral,0.375,Alarming levels of sea level rise are predicte...,neutral,neutral,neutral,agrees,agrees,...,s1,NaN,NaN,NaN,1.0,0.95,A,A,agrees,agrees
2,2,2,neutral,0.000,"Over the past several years, the United States...",neutral,neutral,neutral,neutral,neutral,...,s2,NaN,NaN,NaN,0.0,1.00,B,B,neither agrees nor disagrees,neither agrees nor disagrees
3,3,3,agrees,1.000,Global warming is happening and it will be dan...,agrees,agrees,agrees,agrees,agrees,...,s3,NaN,NaN,NaN,1.0,0.95,A,A,agrees,agrees
4,4,4,neutral,0.000,Some icebergs are cute.,neutral,neutral,neutral,neutral,neutral,...,s4,NaN,NaN,NaN,0.2,0.95,B,B,neither agrees nor disagrees,neither agrees nor disagrees
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,2295,2295,neutral,0.375,"Agriculture, in general, is responsible for a ...",neutral,agrees,neutral,neutral,agrees,...,t5,0.073612,0.242469,0.683919,0.8,0.80,A,A,agrees,agrees
2296,2296,2296,agrees,0.750,"Hiring a White House ""climate change czar"" wou...",agrees,disagrees,agrees,agrees,agrees,...,t6,0.000580,0.998617,0.000803,0.8,0.80,A,B,neither agrees nor disagrees,agrees
2297,2297,2297,neutral,0.250,More carbon is being released than stored.,neutral,neutral,neutral,neutral,agrees,...,t7,0.006119,0.168674,0.825207,0.9,0.90,A,A,agrees,agrees
2298,2298,2298,disagrees,-0.750,New laws are needed to crack down on climate a...,disagrees,disagrees,disagrees,agrees,disagrees,...,t8,0.003142,0.693515,0.303342,1.0,0.80,C,C,disagrees,disagrees


In [10]:
import numpy as np
gpt_confidence = raw_data["confidence_in_prediction_gpt-4o"].to_numpy()
gpt_raw_label = raw_data["label_gpt4o"].to_numpy()
nan_indices = list(np.where(pd.isna(gpt_confidence))[0]) + list(np.where(pd.isna(gpt_raw_label))[0])
good_indices = list(set(range(len(raw_data))) - set(nan_indices))
gold_label = [to_label[label] for label in raw_data["MACE_pred"].to_numpy()[good_indices]]

gpt_label = np.array([to_label[label] for label in raw_data["label_gpt4o"].to_numpy()[good_indices]])
gpt_confidence = raw_data["confidence_in_prediction_gpt-4o"].to_numpy()[good_indices]
data_entry = raw_data["sentence"].to_numpy()[good_indices]
contains_affirming_device = raw_data['sentence'].str.contains(pattern, case=False, regex=True).to_numpy()[good_indices]

data = pd.DataFrame({
    "data_entry" : data_entry, 
    "gpt_label" : gpt_label, 
    "gold_label" : gold_label, 
    "gpt_confidence" : gpt_confidence, 
    "contains_affirming_device" : contains_affirming_device})

n_data_affirming = len(data[data["contains_affirming_device"] == True])
n_data_not_affirming = len(data[data["contains_affirming_device"] == False])
n_data_agree_affirming = len(data[(data["gold_label"] == 1) & (data["contains_affirming_device"] == True)])
n_data_agree_not_affirming = len(data[(data["gold_label"] == 1) & (data["contains_affirming_device"] == False)])

odds = ((n_data_agree_affirming / n_data_affirming) / (1 - n_data_agree_affirming / n_data_affirming)) / \
    ((n_data_agree_not_affirming / n_data_not_affirming) / (1 - n_data_agree_not_affirming / n_data_not_affirming))

print(f"Odds ratio of gold label: {odds}")

n_data_gpt_agree_affirming = len(data[(data["gpt_label"] == 1) & (data["contains_affirming_device"] == True)])
n_data_gpt_agree_not_affirming = len(data[(data["gpt_label"] == 1) & (data["contains_affirming_device"] == False)])

gpt_odds = ((n_data_gpt_agree_affirming / n_data_affirming) / (1 - n_data_gpt_agree_affirming / n_data_affirming)) / \
    ((n_data_gpt_agree_not_affirming / n_data_not_affirming) / (1 - n_data_gpt_agree_not_affirming / n_data_not_affirming))

print(f"Odds ratio of GPT label: {gpt_odds}")

data


Odds ratio of gold label: 0.7981227050114154
Odds ratio of GPT label: 0.9830415551590843


,data_entry,gpt_label,gold_label,gpt_confidence,contains_affirming_device
0,Global warming is a hoax.,0,0,1.00,False
1,Alarming levels of sea level rise are predicte...,1,0,0.95,False
2,"Over the past several years, the United States...",0,0,1.00,True
3,Global warming is happening and it will be dan...,1,1,0.95,False
4,Some icebergs are cute.,0,0,0.95,False
...,...,...,...,...,...
2294,"Agriculture, in general, is responsible for a ...",1,0,0.80,False
2295,"Hiring a White House ""climate change czar"" wou...",1,1,0.80,False
2296,More carbon is being released than stored.,1,0,0.90,False
2297,New laws are needed to crack down on climate a...,0,0,0.80,False


In [11]:
data.to_csv("data.csv", index=False)